In [ ]:
!pip install patool
!pip install timm
!pip install pydicom
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import patoolib

# Path to the RAR file (change this to your file's path)
rar_file_path = "/content/drive/MyDrive/Colab Notebooks/pratice/archive.zip"

# Directory to extract the contents to
extracted_dir = "/content/drive/MyDrive/Colab Notebooks/pratice/clother"

# Extract the RAR file to the specified directory
patoolib.extract_archive(rar_file_path, outdir=extracted_dir)


patool: Extracting /content/drive/MyDrive/Colab Notebooks/pratice/archive.zip ...
patool: running /usr/bin/7z x "-o/content/drive/MyDrive/Colab Notebooks/pratice/clother" -- "/content/drive/MyDrive/Colab Notebooks/pratice/archive.zip"
patool: ... /content/drive/MyDrive/Colab Notebooks/pratice/archive.zip extracted to `/content/drive/MyDrive/Colab Notebooks/pratice/clother'.


'/content/drive/MyDrive/Colab Notebooks/pratice/clother'

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import os
import re
import requests

import matplotlib.pyplot as plt
import timm

In [ ]:
print("Available Vision Transformer Models: ")
timm.list_models("vit*")

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/pratice/clother'
print(os.listdir(data_dir))
print(len(os.listdir(data_dir)))

['black_dress', 'black_pants', 'black_shirt', 'black_shoes', 'black_shorts', 'blue_dress', 'blue_pants', 'blue_shirt', 'blue_shoes', 'blue_shorts', 'brown_pants', 'brown_shoes', 'brown_shorts', 'green_pants', 'green_shirt', 'green_shoes', 'green_shorts', 'red_dress', 'red_pants', 'red_shoes', 'white_dress', 'white_pants', 'white_shoes', 'white_shorts']
24


In [ ]:
def get_path_names(dir):
  images = []
  for path, subdirs, files in os.walk(data_dir):
    for name in files:
      #print(os.path.join(path, name))
      images.append(os.path.join(path, name))
  return images

In [ ]:
classes = ['black', 'blue', 'brown', 'green', 'white', 'red', 'dress', 'pants', 'shorts', 'shoes', 'shirt']
len(classes)

11

In [ ]:
def encode_label(label, classes_list = classes): #encoding the classes into a tensor of shape (11) with 0 and 1s.
    target = torch.zeros(11)
    for l in label:
      idx = classes_list.index(l)
      target[idx] = 1
    return target


def decode_target(target, threshold=0.5): #decoding the prediction tensors of 0s and 1s into text form
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
          result.append(classes[i])
    return ' '.join(result)

In [ ]:
class myDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = get_path_names(root_dir)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        label = re.findall(r'\w+\_\w+', img_path)[0].split('_')

        return img, encode_label(label)

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean and std values of the Imagenet Dataset so that pretrained models could also be used



transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomCrop(224),
    T.RandomHorizontalFlip(),
    T.RandomRotation(2),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])


In [ ]:
label = re.findall(r'\w+\_\w+', '/content/drive/MyDrive/Colab Notebooks/pratice/clother/black_pants/00a2c27ab296e6c2f7220f38c68ef3d11bdd25bd.jpg')[0].split('_')

In [ ]:
label

['black', 'pants']

In [ ]:
a=encode_label(label)

In [ ]:
a

tensor([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [ ]:
dataset = myDataset(data_dir, transform = transform)
len(dataset)

11385

In [ ]:
def denorm(img_tensors):
    return img_tensors * imagenet_stats[1][0] + imagenet_stats[0][0]

def show_example(img,label):
  plt.imshow(denorm(img).permute(1,2,0))
  print("Label:", decode_target(label))
  print()
  print(label)


In [ ]:
val_percent = int(0.15 * len(dataset))
train_size = len(dataset) - val_percent
val_size = len(dataset) - train_size
train_size, val_size

(9678, 1707)

In [ ]:
train_ds, val_ds = random_split(dataset, [train_size, val_size]) #splitting the dataset for training and validation.
len(train_ds), len(val_ds)

(9678, 1707)

In [ ]:
batch_size = 64
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size * 2)

In [ ]:

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
device = get_default_device()
device

device(type='cpu')

In [ ]:
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)

In [ ]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)


        self.model.head = nn.Linear(self.model.head.in_features, n_classes)
        self.output = nn.Sigmoid()
    def forward(self, x):
        x = self.model(x)

        x = self.output(x)
        return x


In [ ]:
model = ViTBase16(11)
model.to('cpu')

ViTBase16(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
  

In [ ]:
from ignite.contrib.handlers import ProgressBar
from ignite.metrics import Accuracy,Loss, RunningAverage
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


NameError: ignored

In [ ]:
val_metrics = {
    "bce": Loss(criterion),

}

In [ ]:
from ignite.metrics import MultiLabelAccuracy

In [ ]:

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)


In [ ]:
best_loss = float('inf')

# RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

# pbar = ProgressBar()
# pbar.attach(trainer)


@trainer.on(Events.ITERATION_COMPLETED(every=50))
def log_training_loss(engine):
    print(f"Epoch {engine.state.epoch} - Iteration {engine.state.iteration} - Loss: {engine.state.output:.4f}")

@trainer.on(Events.EPOCH_COMPLETED)
def run_evaluation(engine):
    global best_loss
    evaluator.run(val_dl)

    metrics = evaluator.state.metrics
    loss = metrics['bce']
    # acc= metrics['acc']
    print(f"Validation Results - Epoch: {engine.state.epoch} - Loss: {loss:.4f} " )
    if loss < best_loss:
        best_loss = loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pratice/ViT.pth')

trainer.add_event_handler(Events.EPOCH_COMPLETED, run_evaluation)


In [ ]:
trainer.run(train_dl, max_epochs=3)


Epoch 1 - Iteration 100 - Loss: 0.3939
Validation Results - Epoch: 1 - Loss: 0.3397 
Validation Results - Epoch: 1 - Loss: 0.3401 
Epoch 2 - Iteration 200 - Loss: 0.2837
Epoch 2 - Iteration 300 - Loss: 0.2766
Validation Results - Epoch: 2 - Loss: 0.3129 
Validation Results - Epoch: 2 - Loss: 0.3151 


ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

In [ ]:
def predict_image(model, image_path, threshold=0.5):
    # Load and preprocess the image
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)  # Preprocess and move to the device

    # Make predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(img)  # Get the model's outputs

    # Apply threshold and decode predictions
    decoded_predictions = decode_target((predictions > threshold).squeeze().cpu().numpy())

    return decoded_predictions

# Load the trained model
model_path = '/content/drive/MyDrive/Colab Notebooks/pratice/ViT.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Path to the image you want to predict


In [ ]:
from sklearn.metrics import f1_score

# Set the model to evaluation mode
model.eval()

# Initialize lists to store predictions and targets
all_predictions = []
all_targets = []

# Iterate through the validation data loader
with torch.no_grad():
    for images, labels in val_dl:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass to get predictions
        predictions = model(images)

        all_predictions.append(predictions.cpu())
        all_targets.append(labels.cpu())

# Concatenate predictions and targets
all_predictions = torch.cat(all_predictions, dim=0)
all_targets = torch.cat(all_targets, dim=0)

# Apply threshold to predictions
threshold = 0.5
binary_predictions = (all_predictions > threshold).float()

# Calculate F1 score
f1 = f1_score(all_targets, binary_predictions, average='samples')

print(f"F1 Score on Validation Data: {f1:.4f}")


F1 Score on Validation Data: 0.5563


In [ ]:
def decode_target(target, threshold=0.5): #decoding the prediction tensors of 0s and 1s into text form
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
          result.append(classes[i])
    return ' '.join(result)

In [ ]:
image_path_to_predict = '/content/drive/MyDrive/Colab Notebooks/pratice/clother/blue_dress/784067f91cfd1a760e2f08fefe60b0618fd34705.jpg'

# Call the prediction function
# predicted_labels = predict_image(model, image_path_to_predict)
img = Image.open(image_path_to_predict)
img = transform(img).unsqueeze(0).to(device)  # Preprocess and move to the device

    # Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
      predictions = model(img)  # Get the model's outputs

print("Predicted Labels:", predictions[0])


Predicted Labels: tensor([1.7353e-02, 9.6098e-01, 8.5206e-04, 1.6677e-03, 1.9635e-02, 3.7194e-03,
        1.6788e-01, 1.0253e-01, 5.4331e-02, 3.9212e-01, 1.0148e-01])


In [ ]:
decode_target(predictions[0])

'blue'

['black', 'blue', 'brown', 'green', 'white', 'red', 'dress', 'pants', 'shorts', 'shoes', 'shirt']

In [ ]:
# Path to the image you want to predict
image_path_to_predict = '/content/drive/MyDrive/Colab Notebooks/pratice/clother/blue_dress/19dec6751c0f3ad362512021bc298558c0cca3c2.jpg'

# Call the prediction function
predicted_labels = predict_image(model, image_path_to_predict)

print("Predicted Labels:", predicted_labels)


Predicted Labels: blue


In [ ]:
from tqdm import tqdm

In [ ]:
print("Available Vision Transformer Models: ")
timm.list_models("vit*")

In [ ]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()